# Bead detection

Detect $3\mu m$ beads inside cells and measure intensity of the signal on the 
beads in all channels. The bead detection is done using normalized cross 
correlation and the cell segmentation is achieved using cellpose.

Process all images from a nikon nd2 file.

# Connecting to hex
ssh <username>@<login node>

# Start a jupyter notebook on the login node
Don't run heavy computation here but instead use dask to create jobs.
```bash
# change to yout code folder
cd ~/code
#srun --partition gpu --gres=gpu:gtx1080ti -c 32 --pty bash -i
#srun --partition cpu -c 112 --pty bash -i
conda activate puroanalysis
jupyter lab --no-browser --ip=0.0.0.0
```
copy link from the "copy paste URLs" into search bar (click on jupyter remote) and then select kernel

Update the code from github using:

In [36]:
!git clone https://github.com/jboulanger/three-microns-beads-in-cells.git

Cloning into 'three-microns-beads-in-cells'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 10 (delta 1), pack-reused 0
Receiving objects: 100% (10/10), 12.49 KiB | 799.00 KiB/s, done.
Resolving deltas: 100% (1/1), done.


In [37]:
cd three-microns-beads-in-cells/

/lmb/home/jeromeb/code/three-microns-beads-in-cells


In [ ]:
!git pull

Already up to date.


Task exception was never retrieved
future: <Task finished name='Task-699' coro=<Client._gather.<locals>.wait() done, defined at /lmb/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/distributed/client.py:2156> exception=AllExit()>
Traceback (most recent call last):
  File "/lmb/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/distributed/client.py", line 2165, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-700' coro=<Client._gather.<locals>.wait() done, defined at /lmb/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/distributed/client.py:2156> exception=AllExit()>
Traceback (most recent call last):
  File "/lmb/home/jeromeb/miniconda3/envs/imaging/lib/python3.9/site-packages/distributed/client.py", line 2165, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-701' coro=<Client._gather.<loca

## Test on a field of view
Test on a single field of view and visually inspect the results

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import nd2
import beadfinder
import pandas as pd


# Define input / output folders

In [ ]:
src_folder = Path('/cephfs2/mlaub/20230511 bSaporin/')
dst_folder = Path('/cephfs2/jeromeb/userdata/mlaub/20230511 bSaporin/')

print('Source folder accessible?', src_folder.exists())
if dst_folder.exists() is False:
    print('Creating destination folder.')
    dst_folder.mkdir(parents=True)
print('Destination folder accessible?', dst_folder.exists())

List all files and save list in destination folder

In [ ]:
files = src_folder.glob('*.nd2')
filelist = []
for filepath in files:    
    with nd2.ND2File(filepath) as f:                
        if f.ndim==4:
            filelist.append({'folder':filepath.parent,'name':filepath.name,'fov':0})
        else:            
            for k in range(f.shape[0]):
                filelist.append({'folder':filepath.parent,'name':filepath.name,'fov':k})

filelist = pd.DataFrame.from_records(filelist)

filelist.to_csv(dst_folder/'filelist.csv',index=False)

## Sequential processing

In [ ]:
# create tasks
results = [beadfinder.process_row(src_folder, dst_folder, row) for row in filelist.iloc]

# concatenate all results in 1 data frame
cells = pd.concat([c for c,_ in results[0]])
beads = pd.concat([b for _,b in results[0]])

# export the data to cs files
cells.to_csv(dst_folder / 'cells.csv')
beads.to_csv(dst_folder / 'beads.csv')

## Parallel processing

Select one type of partition on the cluster by running one of the two cells:

In [ ]:
## GPU nodes
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster(
     cores = 64,
     memory = '32GB',
     queue = 'gpu',
     processes = 1,
     local_directory = '$SLURM_SCRATCH_DIR',
     shebang = '#!/usr/bin/env tcsh',
     walltime = '10:00:00',
     death_timeout = 150,
     job_extra_directives=['--gres=gpu:4']
)
cluster.adapt(maximum_jobs=30)
client = Client(cluster)

In [ ]:
# CPU nodes
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
cluster = SLURMCluster(
     cores = 32,
     memory = '32GB',     
     queue = 'cpu', 
     processes = 1,
     local_directory = '$SLURM_SCRATCH_DIR',     
     shebang = '#!/usr/bin/env tcsh',     
     walltime = '10:00:00',
     death_timeout = 150,
)
cluster.adapt(maximum_jobs=30)
client = Client(cluster)


Process all the files in parallel

In [ ]:
import dask

# create tasks
tsk = [dask.delayed(beadfinder.process_row)(src_folder, dst_folder, row) for row in filelist.iloc]

# run the tasks
results = dask.compute(tsk)

# concatenate all results in 1 data frame
cells = pd.concat([c for c,_ in results[0]])
beads = pd.concat([b for _,b in results[0]])

# export the data to cs files
cells.to_csv(dst_folder / 'cells.csv')
beads.to_csv(dst_folder / 'beads.csv')


In [ ]:
client.shutdown()

## end

In [ ]:
filepath = '/cephfs2/mlaub/20230511 bSaporin/Slide1_A3.nd2'
#/Users/mlaub/Desktop/iSIM Data/20231012 bSaporin z-fa-fmk/Slide1_A1.nd2'
#filepath = '/cephfs2/mlaub/20230511 bSaporin/Slide1_A3.nd2'
#filepath = '/home/'
#filepath = '/Volumes/cephfs2/mlaub/20230511 bSaporin/Slide1_A3.nd2'

fov = 5

with nd2.ND2File(filepath) as f:
    spacing = f.metadata.channels[0].volume.axesCalibration[::-1]
    array = f.to_dask()
    
spacing[0] = spacing[0] * 0.6

#img = nd2.imread(filepath, dask=True)

#img = array[fov,:,:,0:1000,0:1000].compute()
img = array[fov].compute()
cells_df, beads_df, labels = beadfinder.process_img(img, spacing, cell_stitch_threshold=0.1)
#show_beads(img, pixel_size, t2)

#import tifffile
#tifffile.imwrite(filepath.replace('.nd2', '_label.tif'), labels)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
beadfinder.show_beads(np.log(img[:,1]), spacing, beads_df)
plt.show()

In [ ]:
centers = beadfinder.detect_spheres(img[:, 2], spacing, 3, 0.1, 0.3)

In [ ]:
centers2 = beadfinder.detect_beads(img[:,2], spacing, 3, 0.1, 1.4, 1.33, threshold=0.6)

In [ ]:
tmp1 = beadfinder.create_sphere([30,100,100], spacing, 3, 0.1)
plt.imshow(np.amax(tmp1,1))

In [ ]:
plt.imshow(np.log(np.amax(img[:,2,50:250,50:500],1)))

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(np.log(np.amax(img[:,2],0)))
plt.plot(1/spacing[2]*centers[:,2],1/spacing[1] * centers[:,1], 'w.')


In [ ]:
plt.imshow(np.log(np.amax(img[:,0],0)))
plt.scatter(beads_df["X"] / spacing[1], beads_df["Y"] / spacing[2],c=beads_df["Fraction_inside"],cmap='jet')

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot(beads_df,x='Fraction_inside',y='Mean_intensity_ch3')

### Validation of the bead being inside using the 4th channel
The 4th channel should have lower intensity when inside the cells.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
beads_df['inside'] = beads_df['Fraction_inside'] > 0.9
plt.figure(figsize=(10,5))
plt.subplot(121)
sns.scatterplot(beads_df,x='Fraction_inside',y='Mean_intensity_ch3',hue='inside')
plt.subplot(122)
sns.boxplot(beads_df,x='inside',y='Mean_intensity_ch3')

In [ ]:
import pandas as pd

beads_df['inside'] = beads_df['Fraction_inside'] > 0.9
tmp = pd.melt(beads_df,id_vars=['inside'], value_vars=[f'Mean_intensity_ch{k}' for k in range(4)])
sns.boxplot(beads_df,x='inside',y='Mean_intensity_ch3')
#sns.scatterplot(tmp[],x='inside',y='value')
#sns.boxplot(pd.wide_to_long(beads_df,['Mean_intensity_ch{k}' for k in range(4)]), )
#for k in range(4):
#    plt.boxplot(beads_df['Fraction_inside']>0.5, beads_df[f'Mean_intensity_ch{k}'])
tmp

In [ ]:
bead_props = beadfinder.labelprops(labels[1], img, spacing)

In [ ]:
bead_props[0]

In [ ]:
fraction = [np.sum((labels[1]==b['label'])*(labels[0]>0)) / np.sum((labels[1]==b['label'])) for b in bead_props]

In [ ]:
plt.plot(fraction, [b['mean intensity'][3] for b in bead_props],'.')

In [ ]:
t2.plot(x='Fraction_inside',y='Mean_intensity_ch3',kind='scatter')

In [ ]:
template = beadfinder.create_sphere([30,100,100], spacing, radius, 0.1)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
spacing = [0.33,0.11,0.11]
shape = [32,100,100]
radius = [1.5,1.5]
#g = np.meshgrid(*[np.arange(n) for n in shape], indexing="ij")
#d = np.sqrt(sum([((x - n / 2) * p) ** 2 for x, n, p in zip(g, shape, spacing)]))
#b = np.exp(-0.5 * np.square(np.abs(d - radius) / thickness)) + 0.5 * (d < radius)
plt.imshow(np.amax(img,2)[0])

In [ ]:
1.33/1.518, 0.2/0.3, 50/100

In [ ]:
1.4/1.33

In [ ]:
x = np.fft.fftfreq(256, 1)
y = np.zeros(x.shape)
y[np.abs(x)>1e-8] = j1(x[np.abs(x)>1e-8]) / (x[np.abs(x)>1e-8])
plt.plot(np.real(np.fft.fft(y)))


In [ ]:
from scipy.special import j1
shape = [32,512,512]
spacing = [1,1,1]
p = beadfinder.compute_pinhole_otf(shape,spacing, 12)
plt.imshow(np.real(np.fft.fftshift(np.fft.fftn(p[0]))))
p.shape

In [ ]:
1.518/1.37

In [ ]:
np.median(crop)

In [ ]:

from scipy.ndimage import maximum_filter
crop = img[:-3,2,170:240,160:230].astype(float)
#crop = img[:,2,220:350,100:200].astype(float)
shape = crop.shape
spacing = [0.3*0.6,0.11,0.11]


#b = beadfinder.create_sphere(shape, spacing, 1.5, 0.5)
bhat = beadfinder.create_confocal_bead(shape, spacing, 3, 0.5, 1.4, 1.33, 0.5, 0.5, 0.5, 1)
b = np.maximum(0,np.real(np.fft.fftn(bhat)))
xc0 = beadfinder.normzxcorr(b,crop)
#s1 = np.sqrt(np.real(np.fft.ifftn(np.conj(bhat)*bhat)))
#s2 = np.sqrt(np.real(np.fft.ifftn(np.conj(np.fft.fftn(crop))*np.fft.fftn(crop))))
#xc0 = np.real(np.fft.ifftn(np.conj(bhat)*np.fft.fftn(crop-100)))
#xc0 = xc0 


def wiener(f,y,s):    
    return ftconv(np.conj(f) / (s + np.abs(f)**2), y)

xc1 = wiener(bhat,crop,0.01)

def ftconv(f,x):
    return np.real(np.fft.ifftn(f*np.fft.fftn(x)))

def deconvloc(f,y,s,niter):    
    x = wiener(f,y,0.01)
    for _ in range(niter):
        x = x * ftconv(np.conj(f), crop / np.maximum(0.1, ftconv(f, x)))
        x = np.maximum(0.0, x - s*x.max())
    return x

xc2 = deconvloc(bhat, crop, 0.001, 30)

footprint = [2 * 1.5 / n for n in spacing]
zyx = np.argwhere(
    np.logical_and(xc2 > xc2.max() * 0.2, xc2 == maximum_filter(xc2, footprint))
).astype(float)


plt.figure(figsize=(20,20))
plt.subplot(241)
plt.imshow(np.log(1e-3+np.fft.fftshift(b[:,0,:])))
plt.subplot(242)
plt.imshow(np.log(crop[:,shape[1]//2,:]),cmap='Greens')
plt.imshow(np.log(1e-3+np.fft.fftshift(b[:,0,:])),cmap='Reds',alpha=0.6)
plt.subplot(243)
plt.imshow(np.amax(crop,0))
plt.subplot(244)
plt.imshow(np.amax(xc0,0))
plt.subplot(245)
plt.imshow(np.amax(xc1,0))
plt.subplot(246)
plt.imshow(np.amax(xc2,0))
plt.plot(zyx[:,2],zyx[:,1],'ro')
#plt.imshow(np.abs(xc[25]))


In [ ]:
xc2

In [ ]:
viewer = napari.view_image(crop)
viewer.add_image(np.abs(xc))

In [ ]:
print(f'Expected bead volume is {(4/3*3.14156*radius**3)}')
t2.head()

## Process all field of view
We loop over the field of view to process all the images

In [ ]:
filepath = '/cephfs2/mlaub/Example images/A3.nd2'
radius = 1.5
pixel_size = [0.3,0.11,0.11]
img = nd2.imread(filepath, dask=True)
cells = []
beads = []
for k, pos in enumerate(img):
    C,B,label = process_img(pos.compute(), pixel_size, radius)
    # add the index of the field of view
    C['FOV'] = k
    B['FOV'] = k
    cells.append(C)
    beads.append(B)

beads = pd.concat(beads)
cells = pd.concat(cells)

## Processing on the HPC cluster
Connect to hex, activate the environment and start jupyter from hex:
```
ssh hex
conda activate imaging
jupyter lab --no-browser --ip=0.0.0.0
```
Finally, connect to the remote kernel from visual code. We use now dask to distribute the computing across the nodes of the cluster.


In [ ]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, progress
import dask

cluster = SLURMCluster(
     cores=16,
     memory='32GB',
     processes = 1,
     queue = 'cpu',   
     local_directory='$SLURM_SCRATCH_DIR',
     death_timeout=150,
     shebang='#!/usr/bin/env tcsh',     
     walltime='10:00:00',
)

client = Client(cluster)
client

In [ ]:
print(cluster.job_script())

In [ ]:
def fun(x):
    return x+1
tsk = [dask.delayed(fun)(x)  for x in range(10)]


In [ ]:
cluster.scale(10)
#dask.compute(tsk)

In [ ]:
import dask

@dask.delayed
def delayed_process_img(img, pixel_size, radius, fov_idx):
    try :
        C, B, labels = process_img(img, pixel_size, radius)
        C['FOV'] = fov_idx
        B['FOV'] = fov_idx
        return C, B
    except :
        return None, None

filepath = '/cephfs2/mlaub/20230511 bSaporin/Slide1_B3.nd2'
radius = 1.5
pixel_size = [0.3,0.11,0.11]
img = nd2.imread(filepath, dask=True)
cluster.scale(len(img))
tsk = [delayed_process_img(pos, pixel_size, radius, idx) for idx,pos in enumerate(img)]
result = dask.compute(tsk)
cluster.scale(0) # this frees the workers

In [ ]:
cells = pd.concat([r[0] for r in result[0] ])
beads = pd.concat([r[1] for r in result[0] ])

## Export the results
Export the results as CSV files

In [ ]:
# next to the file
beads.to_csv(filepath.replace('.nd2','_beads.csv'))
cells.to_csv(filepath.replace('.nd2','_cells.csv'))
# locally
#beads.to_csv('beads.csv')
#cells.to_csv('cells.csv')

In [ ]:
cluster.scale(0)

## Graph
Get insight from the results by making a figure.

In [ ]:
import pandas as pd
beads = pd.read_csv('beads.csv')
beads.head()

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
c1 = 'Mean_intensity_ch1'
c2 = 'Mean_intensity_ch2'
inside = beads['Fraction_inside'] > 0.75
x = np.sqrt(beads[c1][inside])
y = np.sqrt(beads[c2][inside])
X,Y = np.mgrid[x.min():x.max():100j, y.min():y.max():100j]
kernel = stats.gaussian_kde(np.vstack([x,y]))
Z = kernel(np.vstack([X.ravel(),Y.ravel()])).reshape(X.shape)
#plt.imshow(np.rot90(Z),extent=[x.min(),x.max(),y.min(),y.max()])
plt.contour(X,Y,Z)
plt.plot(x,y,'k.',ms=5,alpha=0.1)
plt.xlabel(c1)
plt.ylabel(c2)
plt.title('Intensity of beads inside cells')

In [ ]:
plt.hist(beads['Volume'])
plt.xlabel('Volume [$\mu m^3$]')
plt.ylabel('Count')
plt.title('Distribution of the volume of the beads')

In [ ]:
#filepath = '/cephfs2/mlaub/20230511 bSaporin/Slide1_A3.nd2'
filepath = '/Volumes/cephfs2/mlaub/20230511 bSaporin/Slide1_A3.nd2'
tbl_beads = pd.read_csv(filepath.replace('.nd2','_beads.csv'))
filepath = '/Users/mlaub/Desktop/Slide1_A3.nd2'

import nd2
radius = 1.5
pixel_size = [0.3,0.11,0.11]
img = nd2.imread(filepath, dask=True)
fov = 1
img = img[fov,:,:,0:500,0:500].compute()
print(img.shape)
print(len(tbl_beads))
tbl_beads = tbl_beads[tbl_beads['FOV']==fov]
print(len(tbl_beads))
#tbl_beads = tbl_beads.iloc[0:10]

tmp  = np.squeeze(img[:,2,:,:])

beads = bead_control(tmp, pixel_size, tbl_beads, 0.5)

plt.imshow(np.amax(tmp,0),cmap='Greens')
plt.imshow(np.amax(beads,0),cmap='Reds')


#import napari
#viewer = napari.view_image(img,scale=pixel_size,depiction='volume',gamma=0.5)
#viewer.add_image(beads,contrast_limits=[0.1,0.5],scale=pixel_size,colormap='red',depiction='volume')
#colors = np.random.rand(len(centers[I<250]), 3)
#viewer.add_points(centers[I<250],size=3,shading='spherical',edge_width=0,face_color=colors)

In [ ]:
plt.imshow(np.amax(tmp,0),cmap='Greens')
plt.imshow(np.amax(beads,0),cmap='Reds')